In [1]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import VarClass
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  running_conf = Struct(backend="numba", signature=True, cache =True, float_precision="single")
  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P)

In [2]:
dim = 2
mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
#mesh_file = "/home/ayoub.hamou/mesh/square.msh"
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
domain, ne, u, v, w, P = init(dim=dim, mesh_path=mesh_file)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 105826
Number of Vertices: 53314
Local domain contruction ...


In [3]:
#PETScKrylovSolver
from manapy.solvers.ls import PETScKrylovSolver

# put this code to PETScKrylovSolver init function so bypass the import of petsc4py
"""
    # try_imports(['import petsc4py',],
    #             'cannot import petsc4py solver!')
    
    # from petsc4py import PETSc as petsc
    
    #self.petsc = petsc
    self.ksp   = None
    
    self.converged_reasons = {}                                                                                                                                                                             
    # for key, val in six.iteritems(petsc.KSP.ConvergedReason.__dict__):                                                                                                                                 
    #     if isinstance(val, int):                                                                                                                                                                       
    #         self.converged_reasons[val] = key    
"""

conf = Struct(reuse_mtx=True, scheme='diamond', verbose=False, precision="single")
L = PETScKrylovSolver(domain=domain, var=P, conf=conf)

SetUp the Linear system ...


In [4]:
# args list
#? The order is imported of the argument list

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    ne,
    domain,
    ne.domain._cells,
    L,
    L.var
])

args = [
  L.domain.faces.cellid,
  L.domain.faces.nodeid,
  L.domain.nodes.oldname,
  L.domain.cells.volume,
  L.domain.nodes.ghostcenter,
  L.domain.cells.loctoglob,
  L.domain.faces.param1,
  L.domain.faces.param2,
  L.domain.faces.param3,
  L.domain.faces.param4,
  L.domain.Pbordnode,
  L.domain.Pbordface,
  L.rhs0,
  L.var.BCdirichlet,
  L.domain.faces.ghostcenter,
  L.matrixinnerfaces,
  L.domain.halofaces,
  L.var.dirichletfaces,
]



---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [10]:
def find_duplicates(arr):
    unique_elements, counts = np.unique(arr, return_counts=True)
    duplicates = unique_elements[counts > 1]
    return duplicates

#? There are duplicate elements on host_cellfid.
#? Data race; use atomic operations to prevent it.
# host_cellfid
find_duplicates(args[0][:, 0])

VarClass([     0,      1,      2, ..., 105765, 105793, 105802],
         dtype=int32)

In [11]:
from manapy.ast.functions2d import get_rhs_loc_2d as cpu_function
from manapy.cuda.manapy.ast.cuda_functions2d import get_kernel_get_rhs_loc_2d as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile get_rhs_loc_2d to cpu => signature=(int32[:,:], int32[:,:], int32[:], float32[:], float32[:,:,:], int32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:], float32[:,:], int32[:], int32[:], int32[:])
compile device_search_element to cuda => signature=int32(int32[:], int32)
compile kernel_get_rhs_loc_2d to cuda => signature=void(int32[:,:], int32[:,:], int32[:], float32[:], float32[:,:,:], int32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], float32[:], int32[:], float32[:,:], int32[:], int32[:], int32[:])
compile kernel_assign to cuda => signature=void(float32[:], float32)


In [12]:
numba_fun(*args)
%timeit numba_fun(*args)

1.45 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_get_rhs_loc_2d.<locals>.kernel_get_rhs_loc_2d at 0x7f61001ef9d0>) is called
VarClass<int32, (159139, 2)>
VarClass<int32, (159139, 3)>
VarClass<int32, (53314,)>
VarClass<float32, (105826,)>
VarClass<float32, (53314, 2, 5)>
VarClass<int32, (105826,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (159139,)>
VarClass<float32, (53314,)>
VarClass<float32, (159139,)>
VarClass<float32, (105826,)>
VarClass<int32, (0,)>
VarClass<float32, (159139, 3)>
VarClass<int32, (158339,)>
VarClass<int32, (0,)>
VarClass<int32, (0,)>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


807 µs ± 93.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
# verify

def verify(a, decimal, plot):
  b = VarClass.to_device(a)
  if plot == True:
    plt.plot(a, label="cpu")
    plt.plot(b, label="gpu")
    plt.legend()
    plt.show()
  np.testing.assert_almost_equal(a, b, decimal=decimal)


for i, arg in enumerate(args):
  print(f"{i + 1} => test arg {arg}")
  verify(arg, decimal=2, plot=False)


1 => test arg [[ 52584     -1]
 [ 53956     -1]
 [ 14290     -1]
 ...
 [  6880  15456]
 [ 65167  69113]
 [104411 105736]]
2 => test arg [[    4     3     2]
 [    5     4     2]
 [    6     5     2]
 ...
 [53313 25518     2]
 [53313 32689     2]
 [53313 48839     2]]
3 => test arg [1 2 2 ... 0 0 0]
4 => test arg [1.2881813e-05 1.3806196e-05 1.2721423e-05 ... 6.8381842e-06 5.7093107e-06
 5.7664242e-06]
5 => test arg [[[ 3.0208819e-03 -1.4129439e-03  5.2585000e+04  4.0000000e+00
    5.9800000e+02]
  [-1.3542156e-03  3.0796106e-03  5.2583000e+04  1.0000000e+00
    6.0000000e+02]]

 [[ 1.0013542e+00  3.0796106e-03  5.2590000e+04  2.0000000e+00
    3.9800000e+02]
  [ 9.9697906e-01 -1.4129439e-03  5.2589000e+04  4.0000000e+00
    4.0000000e+02]]

 [[ 9.9692041e-01  1.0013542e+00  5.2588000e+04  3.0000000e+00
    1.9800000e+02]
  [ 1.0014129e+00  9.9697906e-01  5.2587000e+04  2.0000000e+00
    2.0000000e+02]]

 ...

 [[-1.0000000e+00 -1.0000000e+00 -1.0000000e+00 -1.0000000e+00
   -1.0000000e